## Analysis of high- and low-variance PCs in velocity space during rotation (panel B)

In [ ]:
%matplotlib inline
from definitions import ROOT_DIR
import os
import numpy as np
from sklearn.decomposition import PCA
import matplotlib.pyplot as plt
import matplotlib
from functions_notebook import make_parallel_envs,set_config,cross_project_kin,plot_cross_projection,mean_ratio
import pickle
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis as LDA
from sklearn.model_selection import train_test_split
from stable_baselines3.common.vec_env import VecNormalize
from sb3_contrib import RecurrentPPO
from envs.environment_factory import EnvironmentFactory
import pandas as pd
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import KFold, cross_val_score

## Load the rollouts

In [ ]:
cw_path = os.path.join(ROOT_DIR, "data", "rollouts", "final_model_500_episodes_activations_info_small_variations_cw", "data.hdf")
rollouts_cw = pd.read_hdf(cw_path)
ccw_path = os.path.join(ROOT_DIR, "data", "rollouts", "final_model_500_episodes_activations_info_small_variations_ccw", "data.hdf")
rollouts_ccw = pd.read_hdf(ccw_path)
rollouts_df = pd.concat((rollouts_cw, rollouts_ccw)).reset_index()

In [ ]:
def average_by_timestep(vec, timesteps):
    out_vec = []
    for ts in sorted(np.unique(timesteps)):
        out_vec.append(np.mean(vec[timesteps == ts], axis=0))
    return np.vstack(out_vec)

In [ ]:
def measure_tangling(data):
    derivative = np.gradient(data,axis=0) * 40  # sample frequency

    # epsilon = 0.1*np.mean(np.linalg.norm(data,axis=1))
    epsilon = 1e-10 # * np.mean(np.linalg.norm(data, axis=1))
    # epsilon = 1e-1

    Q_all = []
    for t in range(derivative.shape[0]):
        Q = (np.linalg.norm(derivative[t] - derivative, axis=1)**2) / (epsilon + np.linalg.norm(data[t] - data, axis=1)**2)
        Q = np.max(Q)
        Q_all.append(Q)
    
    return np.mean(Q_all)  # as per definition

In [ ]:
# PCA plots of different component ranges
num_muscles = 39
num_joints = 23
muscle_act = np.vstack(rollouts_df.muscle_act)
pos = np.vstack(rollouts_df.observation)[:, :num_joints]

pos_pc_range_list = [(0, 3), (5, 8), (12, 15), (20, 23)]
muscle_act_pc_range_list = [(0, 3), (23, 26), (36, 39)]
cmap_list = ["Reds", "Blues"]
dir_list = ["cw", "ccw"]
label_list = ["Clockwise", "Counter-clockwise"]
data_name_list = ["joint_pos", "muscle_act"]

for data, pc_range_list, data_name in zip([pos, muscle_act], [pos_pc_range_list, muscle_act_pc_range_list], data_name_list):
    pca = PCA(n_components=data.shape[1])
    out = pca.fit_transform(data)

    for pc_range in pc_range_list:
        fig = plt.figure(figsize=(4, 4))
        ax = fig.add_subplot(projection="3d")

        tangling_list = []
        for cmap_name, direction, label in zip(cmap_list, dir_list, label_list):
            out_direction = out[rollouts_df.task == direction]
            cmap = matplotlib.colormaps[cmap_name]
            color_list = [cmap(i) for i in np.linspace(0.5, 1, 200)]    
            colors = [color_list[idx] for idx in rollouts_df.step[rollouts_df.task == direction]]
            plot_mat = out_direction[:, pc_range[0]:pc_range[1]]
            # ax.scatter(plot_mat[:, 0], plot_mat[:, 1], plot_mat[:, 2], alpha=0.05, s=0.2, c=colors)
            mean_traj = average_by_timestep(plot_mat, rollouts_df.step[rollouts_df.task == direction])
            tangling_list.append(measure_tangling(mean_traj))
            ax.scatter(mean_traj[:, 0], mean_traj[:, 1], mean_traj[:, 2], c=color_list, label=label)
        print(data_name, "PCs:", pc_range, "Tangling:", np.mean(tangling_list))
        # cmap = matplotlib.colormaps["Blues"]
        # color_list = [cmap(i) for i in np.linspace(0.3, 1, 200)]    
        # colors = [color_list[idx] for idx in rollouts_df.step[rollouts_df.task == "ccw"]]
        # ax.scatter(out_ccw[:, -3], out_ccw[:, -2], out_ccw[:, -1], alpha=0.05, s=0.2, c=colors)
        # mean_traj = average_by_timestep(out_ccw[:, -3:], rollouts_df.step[rollouts_df.task == "ccw"])
        # ax.scatter(mean_traj[:, 0], mean_traj[:, 1], mean_traj[:, 2], c=color_list)
        ax.xaxis.set_pane_color((1.0, 1.0, 1.0, 0.0))
        ax.yaxis.set_pane_color((1.0, 1.0, 1.0, 0.0))
        ax.zaxis.set_pane_color((1.0, 1.0, 1.0, 0.0))
        ax.xaxis._axinfo["grid"]['color'] =  (1,1,1,0)
        ax.yaxis._axinfo["grid"]['color'] =  (1,1,1,0)
        ax.zaxis._axinfo["grid"]['color'] =  (1,1,1,0)
        # ax.set_xlim((-.0025, .0025))
        # ax.set_ylim((-.0025, .0025))
        # ax.set_zlim((-.003, .003))
        # ax.set_title(title + " - high variance PCs")
        # ax.view_init(25, 45)
        ax.view_init(30, 45)
        ax.set_xlabel(f"\n\nPC {pc_range[0] + 1}", fontsize=12)
        ax.set_ylabel(f"\n\nPC {pc_range[0] + 2}", fontsize=12)
        ax.set_zlabel(f"\n\nPC {pc_range[0] + 3}", fontsize=12)
        ax.set_box_aspect(aspect=None, zoom=0.7)
        ax.ticklabel_format(style="sci", scilimits=(-2, 2))
        ax.locator_params(axis='both', nbins=4)
        ax.tick_params(axis='both', which='major', labelsize=12)
        ax.tick_params(axis='both', which='minor', labelsize=10)
        # ax.legend()
        out_name = f"pca_{data_name}_components_{'_'.join(str(el) for el in pc_range)}.png"
        fig.savefig(os.path.join(ROOT_DIR, "data", "figures", "panel_2", out_name), format="png", dpi=800, bbox_inches="tight")
        fig.show()

# fig = plt.figure()
# ax = fig.add_subplot(projection="3d")
# ax.scatter(out[:, -3], out[:, -2], out[:, -1], alpha=0.05, s=1, color=color_by_task)
# ax.set_title(title + " - low variance PCs")
# fig.show()

In [ ]:
# Task decoding from the pca trajectories
num_muscles = 39
num_joints = 23
num_episodes_per_direction = 500
max_episode_len = 200
muscle_act = np.vstack(rollouts_df.muscle_act)
pos = np.vstack(rollouts_df.observation)[:, :num_joints]

pos_pc_range_list = [(0, 3), (5, 8), (12, 15), (20, 23)]
pos_pc_range_span = 3
muscle_act_pc_range_list = [(0, 3), (23, 26), (36, 39)]
dir_list = ["cw", "ccw"]
data_name_list = ["joint_pos", "muscle_act"]

for data, pc_range_list, data_name in zip([pos, muscle_act], [pos_pc_range_list, muscle_act_pc_range_list], data_name_list):
    pca = PCA(n_components=data.shape[1])
    out = pca.fit_transform(data)

    for pc_range in pc_range_list:
        X = np.empty((num_episodes_per_direction * len(dir_list), max_episode_len * pos_pc_range_span))
        y = np.empty(num_episodes_per_direction * len(dir_list))
        for dir_idx, dir in enumerate(dir_list):
            for ep_id in range(num_episodes_per_direction):
                step_idx_mask = (rollouts_df.episode == ep_id) & (rollouts_df.task == dir)
                row = out[step_idx_mask, pc_range[0]: pc_range[1]].flatten()
                X[ep_id + dir_idx * num_episodes_per_direction, : len(row)] = row
                y[ep_id + dir_idx * num_episodes_per_direction] = dir_idx
        X = X[:, ~np.all(X[1:] == X[:-1], axis=0)]  # drop constant columns
        
        classification = LogisticRegression()
        cv = KFold(n_splits=5, shuffle=True, random_state=42)
        cv_score = cross_val_score(classification, X, y, cv=cv)

        print(data_name, ", PC range:", pc_range, ", score:", cv_score, ", avg score:", np.mean(cv_score))

In [ ]:
# # PCA of the observations
# num_muscles = 39
# data = np.vstack(rollouts_df.observation)

# pca = PCA(n_components=data.shape[1])
# out = pca.fit_transform(data)
# out_cw = out[tasks == 1]
# out_ccw = out[tasks == 0]
# fig = plt.figure(figsize=(8, 8))
# ax = fig.add_subplot(projection="3d")

# cmap = matplotlib.colormaps["Reds"]
# color_list = [cmap(i) for i in np.linspace(0.3, 1, 200)]    
# colors = [color_list[idx] for idx in rollouts_df.step[rollouts_df.task == "cw"]]
# ax.scatter(out_cw[:, 0], out_cw[:, 1], out_cw[:, 2], alpha=0.1, s=0.2, c=colors)
# mean_traj = average_by_timestep(out_cw[:, :3], rollouts_df.step[rollouts_df.task == "cw"])
# ax.plot(mean_traj[:, 0], mean_traj[:, 1], mean_traj[:, 2], color="r", linewidth=1)

# cmap = matplotlib.colormaps["Blues"]
# color_list = [cmap(i) for i in np.linspace(0.3, 1, 200)]    
# colors = [color_list[idx] for idx in rollouts_df.step[rollouts_df.task == "ccw"]]
# ax.scatter(out_ccw[:, 0], out_ccw[:, 1], out_ccw[:, 2], alpha=0.1, s=0.2, c=colors)
# mean_traj = average_by_timestep(out_ccw[:, :3], rollouts_df.step[rollouts_df.task == "ccw"])
# ax.plot(mean_traj[:, 0], mean_traj[:, 1], mean_traj[:, 2], color="b", linewidth=1)
# ax.xaxis.set_pane_color((1.0, 1.0, 1.0, 0.0))
# ax.yaxis.set_pane_color((1.0, 1.0, 1.0, 0.0))
# ax.zaxis.set_pane_color((1.0, 1.0, 1.0, 0.0))
# ax.xaxis._axinfo["grid"]['color'] =  (1,1,1,0)
# ax.yaxis._axinfo["grid"]['color'] =  (1,1,1,0)
# ax.zaxis._axinfo["grid"]['color'] =  (1,1,1,0)
# # ax.set_xlim((-0.8, 0.8))
# # ax.set_ylim((-.8, .8))
# # ax.set_zlim((-.8, .8))
# # ax.set_title(title + " - high variance PCs")
# ax.view_init(25, 47)
# ax.set_xlabel("PC 1")
# ax.set_ylabel("PC 2")
# ax.set_zlabel("PC 3")


# fig.savefig(os.path.join(ROOT_DIR, "data", "figures", "panel_2", "pca_pos_traj_high_variance.png"), format="png", dpi=600, bbox_inches="tight")
# fig.show()

# # fig = plt.figure()
# # ax = fig.add_subplot(projection="3d")
# # ax.scatter(out[:, -3], out[:, -2], out[:, -1], alpha=0.05, s=1, color=color_by_task)
# # ax.set_title(title + " - low variance PCs")
# # fig.show()

### A. Low-variance PCs : linear classification performance

1. __Go to 2. to directly load the previously-obtained data__\
a. Define the 3 tasks : hold, cw, ccw\
b. Generate and label the velocities for each task\
c. Save the data

In [ ]:
num_ep = 50
num_cond = 3

PATH_TO_NORMALIZED_ENV = os.path.join(
    ROOT_DIR,
    "trained_models/curriculum_steps_complete_baoding_winner/32_phase_2_smaller_rate_resume/env.pkl",
)
PATH_TO_PRETRAINED_NET = os.path.join(
    ROOT_DIR,
    "trained_models/curriculum_steps_complete_baoding_winner/32_phase_2_smaller_rate_resume/model.zip",
)

env_name = "CustomMyoBaodingBallsP2"
render = False

C_hold = set_config(period=1e100,rot_dir=None)
C_cw = set_config(period=5,rot_dir="cw")
C_ccw = set_config(period=5,rot_dir="ccw")

configs = {'hold':C_hold,'cw':C_cw,'ccw':C_ccw}

conds = []

for task in configs :
    envs = make_parallel_envs(env_name, configs[task], num_env=1)
    envs = VecNormalize.load(PATH_TO_NORMALIZED_ENV, envs)
    envs.training = False
    envs.norm_reward = False
    custom_objects = {
        "learning_rate": lambda _: 0,
        "lr_schedule": lambda _: 0,
        "clip_range": lambda _: 0,
    }
    model = RecurrentPPO.load(
            PATH_TO_PRETRAINED_NET, env=envs, device="cpu", custom_objects=custom_objects
        )

    eval_model = model
    eval_env = EnvironmentFactory.create(env_name,**configs[task])
    tot_vel = []
    for n in range(num_ep):
        obs_tot = []
        cum_reward = 0
        lstm_states = None
        obs = eval_env.reset()
        episode_starts = np.ones((1,), dtype=bool)
        done = False
        timestep = 0
        while not done: 
            if render :
                eval_env.sim.render(mode="window")
                
            timestep += 1
            action, lstm_states = eval_model.predict(envs.normalize_obs(obs),
                                                    state=lstm_states,
                                                    episode_start=episode_starts,
                                                    deterministic=True,
                                                    )
                                                        
            obs, rewards, done, info = eval_env.step(action)
            episode_starts = done
            cum_reward += rewards
            obs_tot.append(obs)
        if len(obs_tot) < 200 :
            print("Stopped before 200, task : %s" %task, " number of steps : ",timestep)
            temp = np.zeros((200,86))
            temp[:len(obs_tot)] += obs_tot
            obs_tot = temp
        print('episode %s : '%n,cum_reward)

        # MEASURE JOINT POSITION AND VELOCITY
        hand_positions = np.array(obs_tot)[:,0:23]    
        hand_velocities = np.array([np.diff(pos)/0.0025 for pos in hand_positions.T]).T
        hand_velocities = np.vstack((np.zeros((1,23)),hand_velocities))                                

        conds.append({'task':configs[task],'encoding':task,'reward':cum_reward,'hand velocity':np.array(hand_velocities)})

'''fp = ""
fp_conditions = open(fp, 'wb')
pickle.dump(conds,fp_conditions)
fp_conditions.close()'''

2. Load the velocities and labels for each task

In [ ]:
# Load the file from Basecamp : 'synergies_tasks'
conds = pickle.load(open(os.path.join(ROOT_DIR, "data", "basecamp", "synergies_tasks"), "rb"))
hand_kinematics = np.concatenate([cond['hand velocity'] for cond in conds])
labels = np.array([cond["encoding"] for cond in conds])

3. __Go to 4. to directly load the classification performance vs. number of high-variance PCs removed__\
a. Compute the PCs\
b. Project the velocities on a progressively lower-dimensional subspace\
c. Train a linear classifier to identify the task's identity (Leave-One-Out cross-validation)\
d. Save the data

In [ ]:
n_comp = 23
pca = PCA(n_components=n_comp).fit(hand_kinematics)

In [ ]:
# k = 0
# components = pca.components_[k:, :]
# projected_conds = [{'label':cond['encoding'], 'projected velocity':np.dot(cond['hand velocity']+pca.mean_, components)} for cond in conds]

In [ ]:
hand_kinematics[10]

In [ ]:
k = 1
components = pca.components_[k:, :]
coeffs = pca.transform(hand_kinematics)[:, k:]
projected_kinematics = np.dot(coeffs, components) + pca.mean_
projected_kinematics[10]

In [ ]:
conds[0]["encoding"]

In [ ]:
pca_coeffs = pca.transform(hand_kinematics)

In [ ]:
pca_coeffs.shape

In [ ]:
num_ep = 50
num_cond = 3
performance = []
for k in range(n_comp):
    print(k)
    components = pca.components_[k:, :]
    projected_conds = []
    for cond in conds:
        pca_coeffs = np.dot(cond['hand velocity'] - pca.mean_, components.T)
        # proj = np.dot(pca_coeffs, components) + pca.mean_
        projected_conds.append({'label': cond['encoding'], 'pca_coeffs': pca_coeffs})
    
    X = [d['projected velocity'].flatten() for d in projected_conds]
    y = [d['label'] for d in projected_conds]
    class_performance = []
    for i in range(num_ep):
        x_train, x_test, y_train, y_test = train_test_split(X,y,train_size=num_cond*(num_ep-1),test_size=num_cond)
        lda = LDA().fit(x_train,y_train)
        class_performance.append(lda.score(X=x_test,y=y_test))

    performance.append(np.mean(np.array(class_performance)))
 
'''fp = ""    
fp_perf = open(fp, 'wb')
pickle.dump(performance,fp_perf)
fp.close()'''

4. Load the classification performance vs. number of high-variance PCs removed

In [ ]:
# Load the file from basecamp : 'class_performance_tasks_r'
performance = pickle.load(open(os.path.join(ROOT_DIR, "data", "basecamp", "class_performance_tasks_r"), "rb"))
n_comp=23

5. Plot the classification performance vs. number of high-variance PCs removed

In [ ]:
pc_low_variance = next(x[0] for x in enumerate(pca.explained_variance_ratio_) if x[1] < 0.01)
plt.plot([n for n in range(n_comp)],performance,'-o',linewidth=1,markersize=2,color='black')
plt.axvline(x=pc_low_variance, ymax=0.46,color='r', linestyle='-',linewidth=1,label='1% Variance')
plt.legend(fontsize=21)
plt.xlabel('Number of PCs removed',fontsize=21)
plt.ylabel('Accuracy',fontsize=21)
plt.title('Classification performance',fontsize=21)
plt.yticks(fontsize=21)
plt.xticks(fontsize=21)
plt.subplots_adjust(left=0.21,bottom=0.15)

### B. High-variance PCs : cross-projection similarity

1. Load the velocities and labels for each task (same as in section A.2.a)

In [ ]:
# Load the file from basecamp : 'synergies_tasks'
conds = pickle.load(open('/home/ingster/Bureau/SIL-BigResults/synergies_tasks','rb')) 

hold_velocities = np.concatenate([cond['hand velocity'] for cond in conds if cond['encoding']=='hold'])
cw_velocities = np.concatenate([cond['hand velocity'] for cond in conds if cond['encoding']=='cw'])
ccw_velocities = np.concatenate([cond['hand velocity'] for cond in conds if cond['encoding']=='ccw'])

n_comp = 23
n_highpc = 12 # Considering that the first 12 PCs account for most of the variance = high-variance PCs

2. a. Cross-projection \
b. Quantify the degree of similarity between subpaces by computing average V2/V1\
c. Visualize the degree of similarity between subspaces by plotting cumulative explained variances

In [ ]:
# Cross-projection
cproj_hold_cw = cross_project_kin(vel1=hold_velocities,vel2=ccw_velocities,n_comp=n_comp,n_highpc=n_highpc)
cproj_hold_ccw = cross_project_kin(vel1=hold_velocities,vel2=ccw_velocities,n_comp=n_comp,n_highpc=n_highpc)
cproj_ccw_cw = cross_project_kin(vel1=ccw_velocities,vel2=cw_velocities,n_comp=n_comp,n_highpc=n_highpc)

# Quantify
hold_cw = mean_ratio(n_highpc=n_highpc,cproj=cproj_hold_cw)
hold_ccw = mean_ratio(n_highpc=n_highpc,cproj=cproj_hold_ccw)
ccw_cw = mean_ratio(n_highpc=n_highpc,cproj=cproj_ccw_cw)

hold_cw_r = hold_cw[0]
hold_ccw_r = hold_ccw[0]
ccw_cw_r = ccw_cw[0]

V = {'hold vs. cw':np.round(hold_cw_r,7), 'hold vs. ccw':np.round(hold_ccw_r,7), 'cw vs. ccw':np.round(ccw_cw_r,7)}
print(V)

# Visualize
plot_cross_projection(n_comp=n_comp,cum_1_2=hold_cw[1],cum_2_1=hold_cw[2],label1='Hold on CW',label2='CW on hold')
plot_cross_projection(n_comp=n_comp,cum_1_2=hold_ccw[1],cum_2_1=hold_ccw[2],label1='Hold on CCW',label2='CCW on hold')
plot_cross_projection(n_comp=n_comp,cum_1_2=ccw_cw[1],cum_2_1=ccw_cw[2],label1='CCW on CW',label2='CW on CCW')
